## Login and Security in Python / Flask project
A big portion of any deployed frontend to backend project is to enable User login and security. This project is using JSON Web Tokens (JWT) to enable and guard endpoints according authentication and authorization.   

- The frontend receives the user's ID and password as input and performs an HTTP POST request to log in and authenticate the user.

- The backend validates the credentials against the stored database and prepares a JSON response with the authentication status, e.g., 'Login success' with a 200 status code, or 'Bad request' with a 400 status code.

- If the login is successful, the backend returns a JWT that is stored in the web browser as a cookie. Subsequent requests to the server will include this JWT in the credentials. 

- Python endpoints in the backend that require login credentials will be protected with the @login_required decorator.

- The backend endpoint code can use the return values from the @token_required decorator to control logic based on the user's ID and their authorization level in the system.

### Uses of HTTP requests in Login and Authorization
These requests are common to a user based system.

- Unguarded requests
  - POST request for signing up and creating new users.
  - POST request for logging in and authorizing users.

- Guarded requests, @token_required
  - GET request(s) for fetching user data from the system.
  - PUT request for updating user information, which requires user or admin authentication.
  - DELETE request for removing a user from the database, which requires admin authentication.

## Full Stack and Deployment considerations
The project will be developed on localhost and then deployed to a live server.  Here are some key elements that need solutions and consideration.

- The project will require and easy switch between development URI access and deployment URI access.
- The project will need to pass consistent headers to enable cookies to be passed in request headers.
- The project will need to consider Cross-Orign Resource Share (CORS).  The CORS policies will need to be built into the Python code.
- For deployment CORS policies need to be considered in Nginx configration. 
- For deplyment HTTP methods allowed will need to be added to the Nginx configuration.


## Frontend to Backend code examples
Transitioning from a localhost to a deployed project requires a solid understanding of key system elements.
To have the ability to transition from localhost to deployed project, it is important to understand key elements of the system.  These examples focus on user and login functionalities, which are the starting points of a project and provide valuable insights into common coding techniques and considerations.

### Common URI and Header options
To facilitate consistency in fetch operations, it's best to isolate variables into a single location. This code should be imported and used as a template for every fetch operation.

In [ ]:
// The URI used in fetch is determined by the hostname of the requester.
export var uri;
if (location.hostname === "localhost") {
        uri = "http://localhost:8086";
} else if (location.hostname === "127.0.0.1") {
        uri = "http://127.0.0.1:8086";
} else {
        uri = "https://flask2.nighthawkcodingsociety.com";
}


// A fetch header template should be used to avoid omissions or other errors, 
// (*) the atererisk is assigned, other values are shown for reference. 
export const options = {
    method: 'GET', // *GET, POST, PUT, DELETE, etc.
    mode: 'cors', // no-cors, *cors, same-origin
    cache: 'default', // *default, no-cache, reload, force-cache, only-if-cached
    credentials: 'include', // include, same-origin, omit
    headers: {
        'Content-Type': 'application/json',
    },
};

### Login User, frontend code
This method illustrates login and by using config.js.
- This method illustrates the login process using config.js
- The script retrieves user and password information from the HTML form inputs.
- The script overrides options from config.js by changing the method to POST and adding the form inputs to the request.
- The script handles successful HTTP responses and errors.
  - Upon successful login, the script redirects to a database endpoint.
  - If a failure occurs, the script stays on the page and updates the form with an error message.

In [ ]:
<!-- 
A simple HTML login form that triggers a login action when the button is pressedi, login_user() function.
-->
<!-- Login Form -->
<form action="javascript:login_user()">
    <p><label>
        User ID:
        <input type="text" name="uid" id="uid" required>
    </label></p>
    <p><label>
        Password:
        <input type="password" name="password" id="password" required>
    </label></p>
    <p>
        <button>Login</button>
    </p>
    <p id="error-message" style="color: red;"></p>
</form>


<!-- 
This script handles user authentication. Upon successful authentication, 
it redirects to a page that requires a JWT (JSON Web Token) for access. 
-->
<script>
    // In the context of GitHub Pages, the Liquid site.baseurl variable is used to locate the config.js file.
    import { uri, options } from '{{site.baseurl}}/assets/js/api/config.js';

    // Set the URLs for the endpoints used in this script.
    const url = uri + '/api/users/authenticate';
    const redirect =  uri + '/python/database'; 

    // Method to login user
    function login_user(){

        // Set body to include login data from HTML form
        const body = {
            uid: document.getElementById("uid").value,
            password: document.getElementById("password").value,
        };

        // Modify the options to use the POST method and include the request body.
        const authOptions = {
            ...options, // This will copy all properties from options
            method: 'POST', // Override the method property
            cache: 'no-cache', // Set the cache property
            body: JSON.stringify(body)
        };

        // Clear any previous error messages
        document.getElementById("error-message").textContent = "";

        // Fetch the JWT object from the Web API.
        fetch(url, options)
        .then(response => {
            // trap error response from Web API
            if (!response.ok) {
                const errorMsg = 'Login error: ' + response.status;
                console.log(errorMsg);
                document.getElementById("error-message").textContent = errorMsg;
                return;
            }
            // Success!!!
            // Redirect to Database presentation page
            window.location.href = redirect;
        })
        .catch(error => {
            // Handle any network errors.
            console.log('Possible CORS or service down error: ' + error);
            document.getElementById("error-message").textContent = 'Possible CORS or service down error: ' + error;
        });
    }


</script>

### Login User - backend code
This section illustrates how the backend evaluates a login request.
- The provided Python code demonstrates the key components of building an endpoint.
- The POST method demonstrates the process of validating and obtaining the user ID and password from the JSON request.
- This section illustrates how to query the backend database to obtain a Python user object.
- At various stages, if a check fails, an error code and message are provided.
- Upon successful lookup of a user in the database, a **cookie is generated**.
  - This is returned to the frontend and becomes the **foundation for being authenticated**.
  - The generated cookie from the backend is saved in the browser and is **associated with the web server**. This association is crucial as it allows the server to recognize and authenticate the user in subsequent requests.
  - Note, it is generally best to avoid storing sensitive data in cookies.

In [ ]:
# ... imports

# blueprint for user api endpoints
user_api = Blueprint('user_api', __name__,
                   url_prefix='/api/users')
api = Api(user_api)

# ... more resource code

# api resource class for security
class _Security(Resource):
    # method to authenticate user
    def post(self):
        try:
            body = request.get_json()
            if not body:
                return {
                    "message": "Please provide user details",
                    "data": None,
                    "error": "Bad request"
                }, 400
                
            ''' Get Request Data '''
            uid = body.get('uid')
            if uid is None:
                return {'message': f'User ID is missing'}, 401
            password = body.get('password')
            
            ''' Find User ID in Database '''
            user = User.query.filter_by(_uid=uid).first()
            if user is None or not user.is_password(password):
                return {'message': f"Invalid user id or password"}, 401
            if user:
                try:
                    # Add an expiration date to the JWT token
                    token = jwt.encode(
                        {"_uid": user._uid},
                        current_app.config["SECRET_KEY"],
                        algorithm="HS256"
                    )
                    resp = Response("Authentication for %s successful" % (user._uid))
                    resp.set_cookie("jwt", token,
                            max_age=3600,
                            secure=True,
                            httponly=True,
                            path='/',
                            samesite='None'  # This is the key part for cross-site requests
                            )
                    return resp
                except Exception as e:
                    return {
                        "error": "Something went wrong in cookie creation!",
                        "message": "Failed to generate JWT token: " + str(e)
                    }, 500
            return {
                "message": "Error fetching auth token!",
                "data": None,
                "error": "Unauthorized"
            }, 404
        except Exception as e:
            return {
                    "message": "Something went wrong in data processing!",
                    "error": str(e),
                    "data": None
            }, 500

            
    # building a RESTapi endpoint
    api.add_resource(_Security, '/authenticate')

### Accessing Data, frontend code
Websites often contain public information, such as a home page or the login page we previously discussed. Additionally, websites typically contain secured information that is only available to users who have successfully logged in.  The following frontend code retrieves a list of users from a backend database. This information is only available to users who have an account.
- The HTML shows the column layout for name, id, and age of users in the database.
- The fetch code uses the config.js file. Since this is a GET request, it follows the template exactly. 
- The URL is set to the protected endpoint.
- The fetch request only proceeds if a 200 status code is received. Otherwise, it returns an error message in the HTML table.
- Additionally, the fetch request could fail if the endpoint is down or if the machine accessing the endpoint has network issues, such as a Wi-Fi outage.
- Upon success, the script extracts the body from the response and formats it to match the column layout in the HTML table..

In [ ]:
<!-- HTML table layout for the page. The table is populated by the JavaScript code below. -->
<table>
  <thead>
  <tr>
    <th>Name</th>
    <th>ID</th>
    <th>Age</th>
  </tr>
  </thead>
  <tbody id="result">
    <!-- javascript generated data -->
  </tbody>
</table>

<!-- 
The JavaScript code below fetches user data from an API and displays it in the table. i
It uses the Fetch API to make a GET request to the '/api/users/' endpoint. 
The 'uri' variable and 'options' object are imported from the 'config.js' file.

The script executes sequentially when the page is loaded.
-->
<script type="module">
  // Import 'uri' variable and 'options' object from 'config.js'
  import { uri, options } from '{{site.baseurl}}/assets/js/api/config.js';

  // Set the URL to the 'users' endpoint
  const url = uri + '/api/users/';

  // Get the HTML element where the results will be displayed
  const resultContainer = document.getElementById("result");

  // Make a GET request to the API
  fetch(url, options)
    // response is a RESTful "promise" on any successful fetch
    .then(response => {
      // If the response status is not 200, display an error message
      if (response.status !== 200) {
          const errorMsg = 'Database response error: ' + response.status;
          console.log(errorMsg);
          const tr = document.createElement("tr");
          const td = document.createElement("td");
          td.innerHTML = errorMsg;
          tr.appendChild(td);
          resultContainer.appendChild(tr);
          return;
      }
      // valid response will contain JSON data
      response.json().then(data => {
          console.log(data);
          for (const row of data) {
            // Create a new table row and cells for each piece of data
            // tr and td build out for each row
            const tr = document.createElement("tr");
            const name = document.createElement("td");
            const id = document.createElement("td");
            const age = document.createElement("td");
            // data is specific to the API
            name.innerHTML = row.name; 
            id.innerHTML = row.uid; 
            age.innerHTML = row.age; 
            // this builds td's into tr
            tr.appendChild(name);
            tr.appendChild(id);
            tr.appendChild(age);
            // Append the row to the table
            resultContainer.appendChild(tr);
          }
      })
  })
  // If the fetch request fails (e.g., due to network issues), display an error message
  .catch(err => {
    console.error(err);
    const tr = document.createElement("tr");
    const td = document.createElement("td");
    td.innerHTML = err + ": " + url;
    tr.appendChild(td);
    resultContainer.appendChild(tr);
  });
</script>

### Accessing Data, backend code
The code snippets below illustrate a guarded GET method, where @token_required is the guard.
- The GET method can fail due to the guard, most often resulting in 'Unauthorized' or 'Forbidden' responses.
- The token_required method is the guarding function and handles failure possibilities.
- If token_required is successful, it returns a user object from the database, which is based on the user ID in the token.
- TThe GET method and authentication enable the logged-in user to query all users from the database. The user object returned from the database is not used in this method (self, _). The underscore is a convention indicating that it is not used.
- If the security policy of your application restricts users to only viewing their own data, the method would replace the underscore with 'user' and use it in the database query.
- The GET method returns the protected user list from the database as a result of authenticated access.

In [ ]:
# from user.py file
# ... more import and blueprint code
class UserAPI:        
    class _CRUD(Resource):
        # ... more resource code omitted
        # The @token_required decorator ensures that the user is authenticated before they can access the data.
        @token_required()
        # The get method retrieves all users from the database.
        # The underscore (_) indicates that the current_user is not used in this method.
        def get(self, _): 
            users = User.query.all()    # Query all users from the database
            json_ready = [user.read() for user in users]  # Prepare the data for JSON output
            return jsonify(json_ready)  # Convert the data to a JSON response

    # Add the _CRUD resource to the API at the root endpoint ('/')
    api.add_resource(_CRUD, '/')


# from auth_middlewares.py file
# ... more import code
# The token_required function is a decorator that checks if the user is authenticated.
# If roles are provided, it also checks if the user has the required role.
def token_required(roles=None):
    def decorator(f):
        @wraps(f)
        def decorated(*args, **kwargs):
            token = request.cookies.get("jwt")  # Get the JWT token from the cookies
            # If the token is missing, return an error message
            if not token:
                return {
                    "message": "Authentication Token is missing!",
                    "data": None,
                    "error": "Unauthorized"
                }, 401
            try:
                # Decode the token and get the user's data
                data = jwt.decode(token, current_app.config["SECRET_KEY"], algorithms=["HS256"])
                current_user = User.query.filter_by(_uid=data["_uid"]).first()
                # If the user doesn't exist or doesn't have the required role, return an error message
                if current_user is None:
                    return {
                        "message": "Invalid Authentication token!",
                        "data": None,
                        "error": "Unauthorized"
                    }, 401

                # If the doesn't have the required role, return an error message
                if roles and current_user.role not in roles:
                    return {
                        "message": "Insufficient permissions. Required roles: {}".format(roles),
                        "data": None,
                        "error": "Forbidden"
                    }, 403

            # If there's an error (likely with the token), return an error message
            except Exception as e:
                return {
                    "message": "Something went wrong, likely with the token!",
                    "data": None,
                    "error": str(e)
                }, 500

            # If the user is authenticated and has the required role (if applicable), call the decorated function
            return f(current_user, *args, **kwargs)

        return decorated

    return decorator
